In [1]:
# full-image reconstruction with randomly occluded input

# %% 
import os
from pathlib import Path
import random

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import matplotlib.pyplot as plt

# %matplotlib inline  # uncomment in Jupyter if needed

# -----------------------------
# Reproducibility
# -----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# -----------------------------
# Paths (aligned with previous notebooks)
# -----------------------------
DATA_ROOT = Path("./sts_tooth_data").resolve()
PROCESSED_2D_DIR = DATA_ROOT / "processed_2d"
INDEX_CSV = DATA_ROOT / "sts2d_index.csv"
CHECKPOINT_DIR = DATA_ROOT / "checkpoints"

print("DATA_ROOT      :", DATA_ROOT)
print("PROCESSED_2D   :", PROCESSED_2D_DIR)
print("INDEX_CSV path :", INDEX_CSV)
print("CHECKPOINT_DIR :", CHECKPOINT_DIR)

# -----------------------------
# Device
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [2]:
# %% [markdown]
# ## 1. Build `df_seg` (image–mask pairs)
#
# The index `sts2d_index.csv` has columns:
# - rel_path
# - age_group
# - label_status
# - is_mask
# - pair_id
#
# We will:
# - separate image rows (is_mask == False) and mask rows (is_mask == True),
# - inner-join them on `pair_id` to obtain only pairs that have both image and mask:
#     -> `df_seg` with ~900 rows.
# %%
assert INDEX_CSV.exists(), f"Index CSV not found: {INDEX_CSV}"

df = pd.read_csv(INDEX_CSV)
print("Full index shape:", df.shape)
print(df.head())

df_img = df[df["is_mask"] == False].copy()
df_mask = df[df["is_mask"] == True].copy()

print("\nNumber of image rows:", len(df_img))
print("Number of mask rows :", len(df_mask))

# Keep only the columns we need from the mask df for joining
df_mask_simple = df_mask[["pair_id", "rel_path"]].rename(columns={"rel_path": "mask_rel"})

# Inner join on pair_id to obtain only (image, mask) pairs
df_seg = pd.merge(df_img, df_mask_simple, on="pair_id", how="inner")

print("\nSegmentation dataframe shape:", df_seg.shape)
print(df_seg.head())

print("\nValue counts — age_group in df_seg:")
print(df_seg["age_group"].value_counts())

print("\nValue counts — label_status in df_seg:")
print(df_seg["label_status"].value_counts())

In [3]:
# ## 2. Train/val/test split
#
# We will:
# - Shuffle `df_seg`,
# - Split into:
#     - ~70% train
#     - ~15% val
#     - ~15% test
#
# For simplicity, we perform a random split without explicit stratification.
# (If needed, we could later add stratified splitting based on age_group and label_status.)
# %%
df_seg_shuffled = df_seg.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

n_total = len(df_seg_shuffled)
n_train = int(0.7 * n_total)
n_val = int(0.15 * n_total)
n_test = n_total - n_train - n_val

df_train = df_seg_shuffled.iloc[:n_train].reset_index(drop=True)
df_val   = df_seg_shuffled.iloc[n_train:n_train + n_val].reset_index(drop=True)
df_test  = df_seg_shuffled.iloc[n_train + n_val:].reset_index(drop=True)

print(f"Total samples: {n_total}")
print(f"Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

In [4]:
# %% [markdown]
# ## 3. Helper functions for image and mask paths
#
# We convert `rel_path` and `mask_rel` into actual file paths under
# `processed_2d/` using the same convention as in `preprocessing.ipynb`:
#
# - Images:
#     processed_2d / age_group / label_status / "images" / <filename>
# - Masks:
#     processed_2d / age_group / label_status / "masks"  / <filename>
# %%
def get_image_path_from_row(row: pd.Series) -> Path:
    rel = Path(row["rel_path"])
    age_group = row["age_group"]
    label_status = row["label_status"]

    if age_group not in ("adult", "children"):
        age_group = "unknown"
    if label_status not in ("labeled", "unlabeled"):
        label_status = "unknown"

    img_path = PROCESSED_2D_DIR / age_group / label_status / "images" / rel.name
    return img_path


def get_mask_path_from_row(row: pd.Series) -> Path:
    rel = Path(row["mask_rel"])
    age_group = row["age_group"]
    label_status = row["label_status"]

    if age_group not in ("adult", "children"):
        age_group = "unknown"
    if label_status not in ("labeled", "unlabeled"):
        label_status = "unknown"

    mask_path = PROCESSED_2D_DIR / age_group / label_status / "masks" / rel.name
    return mask_path


def load_image_as_array(path: Path) -> np.ndarray:
    """
    Load a PNG as a grayscale numpy array of shape (H, W), dtype uint8.
    """
    if not path.exists():
        raise FileNotFoundError(f"Image file not found: {path}")
    img = Image.open(path).convert("L")  # grayscale
    return np.array(img)


def load_mask_as_array(path: Path) -> np.ndarray:
    """
    Load a PNG as a mask numpy array of shape (H, W), dtype uint8.
    We will later binarize this mask (foreground vs background).
    """
    if not path.exists():
        raise FileNotFoundError(f"Mask file not found: {path}")
    mask = Image.open(path).convert("L")
    return np.array(mask)


# Quick sanity check
sample_row = df_train.iloc[0]
print("\nSanity check paths:")
print("Image path:", get_image_path_from_row(sample_row))
print("Mask path :", get_mask_path_from_row(sample_row))


In [5]:
# ## 4. Segmentation transforms (Albumentations)
#
# We define:
# - `get_segmentation_train_transform()`:
#     * Resize to (320, 640)
#     * Horizontal flip
#     * Small rotation
#     * Mild brightness/contrast, noise, blur
#     * Normalize (mean=0.5, std=0.25)
#     * Convert to tensor
#
# - `get_segmentation_val_transform()`:
#     * Resize + normalize + tensor
#
# Masks are not normalized by A.Normalize. ToTensorV2 converts them to tensors, 
# and we explicitly binarize with (mask > 0.5) afterward.

# We will binarize them after the transform.
# %%
TARGET_HEIGHT = 320
TARGET_WIDTH = 640

def get_segmentation_train_transform() -> A.Compose:
    return A.Compose(
        [
            A.Resize(height=TARGET_HEIGHT, width=TARGET_WIDTH),
            A.HorizontalFlip(p=0.5),
            A.Rotate(
                limit=5,
                border_mode=cv2.BORDER_REFLECT_101,
                p=0.5,
            ),
            # A.RandomBrightnessContrast(
            #     brightness_limit=0.1,
            #     contrast_limit=0.1,
            #     p=0.5,
            # ),
            # A.GaussNoise(p=0.2),
            # A.GaussianBlur(blur_limit=(3, 5), p=0.2),
            # A.ToFloat(max_value=255.0),
            A.Normalize(mean=(0.5,), std=(0.25,)),
            ToTensorV2(),
        ],
        additional_targets={"mask": "mask"},
    )


def get_segmentation_val_transform() -> A.Compose:
    return A.Compose(
        [
            A.Resize(height=TARGET_HEIGHT, width=TARGET_WIDTH),
            # A.ToFloat(max_value=255.0),
            A.Normalize(mean=(0.5,), std=(0.25,)),
            ToTensorV2(),
        ],
        additional_targets={"mask": "mask"},
    )


train_transform = get_segmentation_train_transform()
val_transform = get_segmentation_val_transform()
print("Segmentation transforms created.")

In [6]:
# ## 5. `DentalSegmentationDataset`
#
# This Dataset:
# - For each row in df_train/df_val/df_test:
#     * loads grayscale image and mask,
#     * applies Albumentations transform (with consistent transform on image+mask),
#     * binarizes mask to {0,1},
#     * returns `(image_tensor, mask_tensor, meta_dict)`.
# %%
class DentalSegmentationDataset(Dataset):
    def __init__(self, df_seg: pd.DataFrame, transform: A.Compose):
        """
        Args:
            df_seg: DataFrame containing image–mask pairs.
            transform: Albumentations transform (for both image and mask).
        """
        self.df = df_seg.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]

        img_path = get_image_path_from_row(row)
        mask_path = get_mask_path_from_row(row)

        img_np = load_image_as_array(img_path)   # (H, W), uint8
        mask_np = load_mask_as_array(mask_path)  # (H, W), uint8

        # Albumentations expects dict with "image" and "mask"
        augmented = self.transform(image=img_np, mask=mask_np)
        img_t = augmented["image"]  # FloatTensor, shape (1, H, W)
        mask_t = augmented["mask"]  # FloatTensor, shape (1, H, W) or (H, W)

        # Ensure mask has shape (1, H, W)
        if mask_t.ndim == 2:
            mask_t = mask_t.unsqueeze(0)

        # Binarize mask to {0,1}
        mask_t = (mask_t > 0.5).float()

        meta = {
            "img_path": str(img_path),
            "mask_path": str(mask_path),
            "pair_id": row["pair_id"],
            "age_group": row["age_group"],
            "label_status": row["label_status"],
        }

        return img_t, mask_t, meta


# Quick sanity check: one sample
train_ds = DentalSegmentationDataset(df_train, transform=train_transform)
img_t, mask_t, meta = train_ds[0]
print("\nSample from train_ds:")
print("Image tensor shape:", img_t.shape)
print("Mask tensor shape :", mask_t.shape)
print("Meta:", meta)


In [7]:
# ## 6. Dataloaders
#
# We create three dataloaders:
# - train_loader
# - val_loader
# - test_loader
#
# Training loader is shuffled, validation/test loaders are not.
# %%
BATCH_SIZE = 4
NUM_WORKERS = 0  # adjust depending on your machine

train_ds = DentalSegmentationDataset(df_train, transform=train_transform)
val_ds   = DentalSegmentationDataset(df_val,   transform=val_transform)
test_ds  = DentalSegmentationDataset(df_test,  transform=val_transform)

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=(device.type == "cuda"),
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=(device.type == "cuda"),
)

test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=(device.type == "cuda"),
)

print("Dataloaders ready:")
print("  train batches:", len(train_loader))
print("  val batches  :", len(val_loader))
print("  test batches :", len(test_loader))


In [8]:
# ## 7. 2D U-Net model (same as in pretrain_MIM.ipynb)
#
# We reuse the same 2D U-Net architecture:
# - in_channels = 1  (grayscale)
# - out_channels = 1 (binary segmentation logit)
#
# The only difference is the interpretation of the output: here it is a
# per-pixel logit for "tooth" vs "background".
# %%
class DoubleConv(nn.Module):
    """(Conv2d -> BN -> ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if mid_channels is None:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with MaxPool then DoubleConv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(in_channels, out_channels),
        )

    def forward(self, x):
        return self.block(x)


class Up(nn.Module):
    """Upscaling then DoubleConv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(
                in_channels, in_channels // 2, kernel_size=2, stride=2
            )
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        # x1: from previous decoder level
        # x2: from encoder skip connection
        x1 = self.up(x1)

        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = nn.functional.pad(
            x1,
            [diffX // 2, diffX - diffX // 2,
             diffY // 2, diffY - diffY // 2],
        )

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    """Final 1x1 convolution to produce output channels"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class UNet2D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, base_channels=32, bilinear=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.bilinear = bilinear
        self.base_channels = base_channels

        self.inc = DoubleConv(in_channels, base_channels)
        self.down1 = Down(base_channels, base_channels * 2)
        self.down2 = Down(base_channels * 2, base_channels * 4)
        self.down3 = Down(base_channels * 4, base_channels * 8)
        self.down4 = Down(base_channels * 8, base_channels * 8)

        self.up1 = Up(base_channels * 16, base_channels * 4, bilinear)
        self.up2 = Up(base_channels * 8, base_channels * 2, bilinear)
        self.up3 = Up(base_channels * 4, base_channels, bilinear)
        self.up4 = Up(base_channels * 2, base_channels, bilinear)

        self.outc = OutConv(base_channels, out_channels)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x,  x3)
        x = self.up3(x,  x2)
        x = self.up4(x,  x1)

        logits = self.outc(x)
        return logits


model = UNet2D(in_channels=1, out_channels=1, base_channels=32, bilinear=True)
model = model.to(device)
print(model)


In [9]:
# ## 8. Load Pretrain weights (optional)
#
# We use the checkpoint from MIM pretraining:
# - `unet2d_mim_pretrained.pth`
#
# Strategy:
# - Build a UNet2D with the same architecture.
# - Load the entire state dict.
# - Optionally re-initialize the final `outc` layer (since the meaning
#   of the output changes from "reconstruction" to "segmentation logit").
#
# By setting `USE_PRETRAINED = False`, you can train a baseline model
# from scratch to compare with the MIM-pretrained initialization.
# %%
USE_PRETRAINED = True  # set to False for a from-scratch baseline

# mim pretrain checkpoint path
# ckpt_path = CHECKPOINT_DIR / "unet2d_mim_pretrained.pth"

# full image pretrain checkpoint path
ckpt_path = CHECKPOINT_DIR / "unet2d_fullimage_pretrained.pth"

if USE_PRETRAINED and ckpt_path.exists():
    state_dict = torch.load(ckpt_path, map_location=device)
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
    print("Loaded MIM-pretrained weights from:", ckpt_path)
    print("Missing keys   :", missing)
    print("Unexpected keys:", unexpected)

    # Optionally re-init final output layer for segmentation
    model.outc = OutConv(model.base_channels, 1).to(device)
    print("Reinitialized final output layer (outc) for segmentation.")
else:
    if USE_PRETRAINED:
        print("WARNING: USE_PRETRAINED=True but checkpoint not found, training from scratch.")
    else:
        print("Training from scratch (no pretrained weights loaded).")


In [10]:
# %% [markdown]
# ## 9. Loss functions and metrics
#
# - Loss:
#     * `BCEWithLogitsLoss` + Dice loss
# - Metrics:
#     * Dice coefficient (foreground)
#     * IoU (Jaccard index)
# %%
bce_loss_fn = nn.BCEWithLogitsLoss()

def dice_loss_from_logits(logits: torch.Tensor, targets: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
    """
    Soft Dice loss computed from logits and target masks in {0,1}.
    Args:
        logits:  (B, 1, H, W)
        targets: (B, 1, H, W) in {0,1}
    """
    probs = torch.sigmoid(logits)
    # Flatten per batch
    probs_flat = probs.view(probs.size(0), -1)
    targets_flat = targets.view(targets.size(0), -1)

    intersection = (probs_flat * targets_flat).sum(dim=1)
    union = probs_flat.sum(dim=1) + targets_flat.sum(dim=1)

    dice = (2.0 * intersection + eps) / (union + eps)
    return 1.0 - dice.mean()


def combined_segmentation_loss(logits: torch.Tensor, targets: torch.Tensor, lambda_dice: float = 0.2) -> torch.Tensor:
    """
    Combined loss: BCEWithLogits + lambda_dice * DiceLoss
    """
    loss_bce = bce_loss_fn(logits, targets)
    loss_dice = dice_loss_from_logits(logits, targets)
    return loss_bce + lambda_dice * loss_dice


def compute_dice_score(logits: torch.Tensor, targets: torch.Tensor, threshold: float = 0.5, eps: float = 1e-6) -> float:
    """
    Compute foreground Dice coefficient from logits and targets in {0,1}.
    Returns a scalar (mean over batch).
    """
    probs = torch.sigmoid(logits)
    preds = (probs >= threshold).float()

    preds_flat = preds.view(preds.size(0), -1)
    targets_flat = targets.view(targets.size(0), -1)

    intersection = (preds_flat * targets_flat).sum(dim=1)
    union = preds_flat.sum(dim=1) + targets_flat.sum(dim=1)

    dice = (2.0 * intersection + eps) / (union + eps)
    return dice.mean().item()


def compute_iou_score(logits: torch.Tensor, targets: torch.Tensor, threshold: float = 0.5, eps: float = 1e-6) -> float:
    """
    Compute IoU (Jaccard index) from logits and targets in {0,1}.
    Returns a scalar (mean over batch).
    """
    probs = torch.sigmoid(logits)
    preds = (probs >= threshold).float()

    preds_flat = preds.view(preds.size(0), -1)
    targets_flat = targets.view(targets.size(0), -1)

    intersection = (preds_flat * targets_flat).sum(dim=1)
    union = preds_flat.sum(dim=1) + targets_flat.sum(dim=1) - intersection

    iou = (intersection + eps) / (union + eps)
    return iou.mean().item()


In [11]:
# ## 10. Optimizer and training hyperparameters
#
# - Optimizer: Adam
# - LR: 1e-4 (smaller than MIM pretraining LR)
# - Epochs: start with 50 as a reasonable number
# - Early stopping: based on validation Dice, with a patience window.
# %%
LR = 5e-5
NUM_EPOCHS = 100
PATIENCE = 10  # epochs without val Dice improvement before early stopping

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
print("Optimizer:", optimizer)


In [12]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from pathlib import Path

# Log path：sts_tooth_data/runs/seg_unet
log_dir = DATA_ROOT / "runs" / "seg_unet"
log_dir.mkdir(parents=True, exist_ok=True)

writer = SummaryWriter(log_dir=str(log_dir))
print("TensorBoard log dir:", log_dir)

In [20]:
# ## 11. Fine-tuning loop (with tqdm + TensorBoard)
#
# For each epoch:
# - Train:
#     * forward on train batches
#     * compute combined segmentation loss
#     * backprop + update
# - Validate:
#     * compute loss, Dice, IoU on val set (no gradient)
# - Track:
#     * best validation Dice
#     * save best model checkpoint
# - Early stopping based on val Dice.

from tqdm.auto import tqdm

CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
# best_ckpt_path = CHECKPOINT_DIR / ("unet2d_seg_mim_best.pth" if USE_PRETRAINED else "unet2d_seg_scratch_best.pth")
best_ckpt_path = CHECKPOINT_DIR / ("unet2d_seg_full_image_best.pth" if USE_PRETRAINED else "unet2d_seg_scratch_best.pth")
best_val_dice = 0.0
epochs_without_improvement = 0

global_step = 0 

for epoch in range(1, NUM_EPOCHS + 1):
    # -------------------------
    # Training phase
    
    # -------------------------
    model.train()
    train_loss_sum = 0.0
    train_batches = 0

    # The tqdm progress bar is wrapped in a train_loader layer.
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d} [train]", leave=False)
    for batch in train_pbar:
        imgs, masks, meta = batch
        imgs = imgs.to(device)   # (B, 1, H, W)
        masks = masks.to(device) # (B, 1, H, W)

        optimizer.zero_grad()

        logits = model(imgs)
        loss = combined_segmentation_loss(logits, masks, lambda_dice=0.2)

        loss.backward()
        optimizer.step()

        train_loss_sum += loss.item()
        train_batches += 1
        global_step += 1

        avg_batch_loss = train_loss_sum / train_batches

        # Update the loss displayed on the progress bar.
        train_pbar.set_postfix({"loss": f"{avg_batch_loss:.4f}"})

        # TensorBoard: Records training loss step by step.
        writer.add_scalar("train/loss_step", loss.item(), global_step)

    avg_train_loss = train_loss_sum / max(1, train_batches)
    writer.add_scalar("train/loss_epoch", avg_train_loss, epoch)

    # -------------------------
    # Validation phase
    # -------------------------
    model.eval()
    val_loss_sum = 0.0
    val_batches = 0
    val_dice_sum = 0.0
    val_iou_sum = 0.0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch:02d} [val]", leave=False)
        for batch in val_pbar:
            imgs, masks, meta = batch
            imgs = imgs.to(device)
            masks = masks.to(device)

            logits = model(imgs)
            loss = combined_segmentation_loss(logits, masks, lambda_dice=0.2)

            val_loss_sum += loss.item()
            val_batches += 1

            # metrics
            batch_dice = compute_dice_score(logits, masks)
            batch_iou  = compute_iou_score(logits, masks)

            val_dice_sum += batch_dice
            val_iou_sum  += batch_iou

            avg_val_loss_tmp = val_loss_sum / max(1, val_batches)
            avg_val_dice_tmp = val_dice_sum / max(1, val_batches)

            # The verification progress bar displays the current average value loss / die.
            val_pbar.set_postfix({
                "val_loss": f"{avg_val_loss_tmp:.4f}",
                "val_dice": f"{avg_val_dice_tmp:.4f}",
            })

    avg_val_loss = val_loss_sum / max(1, val_batches)
    avg_val_dice = val_dice_sum / max(1, val_batches)
    avg_val_iou  = val_iou_sum  / max(1, val_batches)

    # TensorBoard: Records validation metrics by epoch.
    writer.add_scalar("val/loss", avg_val_loss, epoch)
    writer.add_scalar("val/dice", avg_val_dice, epoch)
    writer.add_scalar("val/iou",  avg_val_iou,  epoch)
    writer.flush()

    print(
        f"Epoch [{epoch:02d}/{NUM_EPOCHS}] "
        f"- Train loss: {avg_train_loss:.4f} "
        f"- Val loss: {avg_val_loss:.4f} "
        f"- Val Dice: {avg_val_dice:.4f} "
        f"- Val IoU: {avg_val_iou:.4f}"
    )

    # -------------------------
    # Check for improvement (by val Dice)
    # -------------------------
    if avg_val_dice > best_val_dice:
        best_val_dice = avg_val_dice
        epochs_without_improvement = 0
        torch.save(model.state_dict(), best_ckpt_path)
        print(f"  -> New best val Dice: {best_val_dice:.4f}. Saved checkpoint to {best_ckpt_path}")
    else:
        epochs_without_improvement += 1
        print(f"  -> No improvement in Dice for {epochs_without_improvement} epoch(s).")

    # Early stopping
    if epochs_without_improvement >= PATIENCE:
        print(f"Early stopping triggered at epoch {epoch}.")
        break

writer.close()


In [13]:
# %% [markdown]
# ## 12. Evaluation on test set (using best checkpoint)
#
# We load the best checkpoint (according to validation Dice)
# and evaluate on the held-out test set:
# - Test loss
# - Test Dice
# - Test IoU
# %%

# Load best model for mim image training
# best_ckpt_path = CHECKPOINT_DIR / ("unet2d_seg_mim_best.pth" if USE_PRETRAINED else "unet2d_seg_scratch_best.pth")

# Load best model for full image training
best_ckpt_path = CHECKPOINT_DIR / ("unet2d_seg_full_image_best.pth" if USE_PRETRAINED else "unet2d_seg_scratch_best.pth")

if best_ckpt_path.exists():
    state_dict = torch.load(best_ckpt_path, map_location=device)
    model.load_state_dict(state_dict)
    print("Loaded best segmentation checkpoint from:", best_ckpt_path)
else:
    print("WARNING: Best checkpoint not found, using current model weights.")

model.eval()

test_loss_sum = 0.0
test_batches = 0
test_dice_sum = 0.0
test_iou_sum = 0.0

with torch.no_grad():
    for batch in test_loader:
        imgs, masks, meta = batch
        imgs = imgs.to(device)
        masks = masks.to(device)

        logits = model(imgs)
        loss = combined_segmentation_loss(logits, masks, lambda_dice=0.2)
        test_loss_sum += loss.item()
        test_batches += 1

        batch_dice = compute_dice_score(logits, masks)
        batch_iou  = compute_iou_score(logits, masks)

        test_dice_sum += batch_dice
        test_iou_sum  += batch_iou

avg_test_loss = test_loss_sum / max(1, test_batches)
avg_test_dice = test_dice_sum / max(1, test_batches)
avg_test_iou  = test_iou_sum  / max(1, test_batches)

print("\n=== Test set evaluation ===")
print(f"Test loss: {avg_test_loss:.4f}")
print(f"Test Dice: {avg_test_dice:.4f}")
print(f"Test IoU : {avg_test_iou:.4f}")


In [14]:
# ## 13. Qualitative visualization (optional)
#
# We visualize a few samples from the test set:
# - Input X-ray
# - Ground truth mask
# - Predicted mask (thresholded at 0.5)
# - Overlay
# %%
NUM_VIS_SAMPLES = 6

model.eval()
with torch.no_grad():
    count = 0
    for imgs, masks, meta in test_loader:
        imgs = imgs.to(device)
        masks = masks.to(device)

        logits = model(imgs)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).float()

        for b in range(imgs.size(0)):
            if count >= NUM_VIS_SAMPLES:
                break

            img_np   = imgs[b].cpu().squeeze(0).numpy()
            gt_np    = masks[b].cpu().squeeze(0).numpy()
            pred_np  = preds[b].cpu().squeeze(0).numpy()

            fig, axes = plt.subplots(1, 4, figsize=(16, 4))

            axes[0].imshow(img_np, cmap="gray")
            axes[0].set_title("Input X-ray")
            axes[0].axis("off")

            axes[1].imshow(gt_np, cmap="gray")
            axes[1].set_title("Ground truth mask")
            axes[1].axis("off")

            axes[2].imshow(pred_np, cmap="gray")
            axes[2].set_title("Predicted mask")
            axes[2].axis("off")

            # Overlay prediction on input
            axes[3].imshow(img_np, cmap="gray")
            axes[3].imshow(pred_np, alpha=0.4, cmap="jet")
            axes[3].set_title("Overlay (prediction)")
            axes[3].axis("off")

            plt.tight_layout()
            plt.show()

            print("Image path:", meta["img_path"][b])
            count += 1

        if count >= NUM_VIS_SAMPLES:
            break

In [103]:
images, masks, metas = next(iter(train_loader))

print("images shape:", images.shape)
print("images dtype:", images.dtype)
print("images range:", images.min().item(), "->", images.max().item())

print("masks shape:", masks.shape)
print("masks dtype:", masks.dtype)
print("masks unique:", masks.unique())

print("metas type:", type(metas))
print("metas keys:", metas.keys() if isinstance(metas, dict) else None)


In [ ]:
!tensorboard --logdir sts_tooth_data/runs